In [1]:
%matplotlib inline
import torch
import matplotlib.pyplot as plt
import numpy as np

train_x = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float).reshape(-1,2)
train_y = torch.tensor([[0], [1], [1], [0]], dtype=torch.float)

W1_init = torch.tensor([[10.0, -10.0], [10.0, -10.0]], requires_grad=True)
b1_init =  torch.tensor([[-5.0, 15.0]], requires_grad=True)
W2_init =  torch.tensor([[10.0], [10.0]], requires_grad=True)
b2_init =  torch.tensor([[-15.0]], requires_grad=True)

print(W1_init.shape)
print(W2_init.shape)
print(b1_init.shape)
print(b2_init.shape)

In [ ]:
class XOROperatorModel:
    def __init__(self, W1=W1_init, W2=W2_init, b1=b1_init, b2=b2_init):
        self.W1 = W1
        self.W2 = W2
        self.b1 = b1
        self.b2 = b2

    # Predictor
    def f1(self, x):
        return torch.sigmoid(x @ self.W1 + self.b1)

    def f2(self,x):
        return torch.sigmoid(x @ self.W2 + self.b2)

    def f(self, x):
        return self.f2(self.f1(x))  
    

    def loss(self, x, y):
        return torch.nn.functional.binary_cross_entropy_with_logits(self.f(x),y)

In [ ]:
model = XOROperatorModel()

model2 = XOROperatorModel(
    torch.rand((2,2), requires_grad=True),
    torch.rand((2,1), requires_grad=True),
    torch.rand((1,2), requires_grad=True),
    torch.rand((1,1), requires_grad=True)
)

optimizer = torch.optim.SGD([model.b1, model.W1, model.W2, model.b2], lr=0.1)
optimizer2 = torch.optim.SGD([model2.b1, model2.W1, model2.W2, model2.b2], lr=0.1)
for epoch in range(250000):
    model.loss(train_x, train_y).backward()  
    optimizer.step() 
    optimizer.zero_grad()  
    model2.loss(train_x, train_y).backward()  
    optimizer2.step() 
    optimizer2.zero_grad()  

print("Model 1 W1 = %s, b1 = %s, W2 = %s, b2 = %s, loss = %s" %(model.b1, model.W1, model.W2, model.b2, model.loss(train_x, train_y)))
print("Model 2W1 = %s, b1 = %s, W2 = %s, b2 = %s, loss = %s" %(model2.b1, model2.W1, model2.W2, model2.b2, model2.loss(train_x, train_y)))

/home/hermanoe/.local/lib/python3.8/site-packages/torch/optim/sgd.py:69: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super(SGD, self).__init__(params, defaults)


Model 1 W1 = tensor([[-5.2275, 15.5728]], requires_grad=True), b1 = tensor([[ 10.6046, -10.4728],
        [ 10.6046, -10.4728]], requires_grad=True), W2 = tensor([[10.0442],
        [10.0395]], requires_grad=True), b2 = tensor([[-15.]], requires_grad=True), loss = tensor(0.5060, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Model 2W1 = tensor([[0.8008, 0.6073]], requires_grad=True), b1 = tensor([[ 7.9493,  7.5883],
        [-7.9024, -7.6234]], requires_grad=True), W2 = tensor([[-7.2107],
        [-6.5187]], requires_grad=True), b2 = tensor([[0.9524]], requires_grad=True), loss = tensor(0.6191, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)


In [ ]:
xt =train_x.t()[0]
yt =train_x.t()[1]

fig = plt.figure("Logistic regression: the logical OR operator")

plot1 = fig.add_subplot(111, projection='3d')

plot1_f = plot1.plot_wireframe(np.array([[]]), np.array([[]]), np.array([[]]), color="green", label="$\\hat y=f(\\mathbf{x})=\\sigma(\\mathbf{xW}+b)$")

plot1.plot(xt.squeeze(), yt.squeeze(), train_y[:, 0].squeeze(), 'o', label="$(x_1^{(i)}, x_2^{(i)},y^{(i)})$", color="blue")

plot1_info = fig.text(0.01, 0.02, "")

plot1.set_xlabel("$x_1$")
plot1.set_ylabel("$x_2$")
plot1.set_zlabel("$y$")
plot1.legend(loc="upper left")
plot1.set_xticks([0, 1])
plot1.set_yticks([0, 1])
plot1.set_zticks([0, 1])
plot1.set_xlim(-0.25, 1.25)
plot1.set_ylim(-0.25, 1.25)
plot1.set_zlim(-0.25, 1.25)

table = plt.table(cellText=[[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0]],
                  colWidths=[0.1] * 3,
                  colLabels=["$x_1$", "$x_2$", "$f(\\mathbf{x})$"],
                  cellLoc="center",
                  loc="lower right")


plot1_f.remove()
def plotModel(model, plot, color):
    x1_grid, x2_grid = np.meshgrid(np.linspace(-0.25, 1.25, 10), np.linspace(-0.25, 1.25, 10))
    y_grid = np.empty([10, 10])
    for i in range(0, x1_grid.shape[0]):
        for j in range(0, x1_grid.shape[1]):
            y_grid[i, j] = model.f(torch.tensor([[(x1_grid[i, j]),  (x2_grid[i, j])]], dtype=torch.float))
    plot1_f = plot.plot_wireframe(x1_grid, x2_grid, y_grid, color=color)

plotModel(model, plot1, "green")
plotModel(model2, plot1, "orange")

fig.canvas.draw()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/hermanoe/.local/lib/python3.8/site-packages/mpl_toolkits/mplot3d/proj3d.py:136: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  return np.array([xs, ys, zs, np.ones_like(xs)])
/home/hermanoe/.local/lib/python3.8/site-packages/mpl_toolkits/mplot3d/proj3d.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([xs, ys, zs, np.ones_like(xs)])
